In [4]:
import pandas as pd
import os
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline
print ("plt version:[{}]".format(matplotlib.__version__))
print ("gym version:[{}]".format(gym.__version__))

plt version:[3.4.3]


NameError: name 'gym' is not defined

dataset :https://people.sc.fsu.edu/~jburkardt/datasets/tsp/tsp.html

In [5]:
tsp_data_txt = open('tsp_dataset.txt')

In [6]:
lines = tsp_data_txt.readlines()

In [7]:
coordinate_maps = np.zeros((len(lines),len(lines)))

In [8]:
for i,line in enumerate(lines):
    for j,element in enumerate((line.split('\n')[0].split('       '))):
        if(element == ''):
            continue
        
        coordinate_maps[i][j-1] = int(element.strip())

In [9]:
coordinate_maps

array([[ 0., 29., 82., 46., 68., 52., 72., 42., 51., 55., 29., 74., 23.,
        72., 46.],
       [29.,  0., 55., 46., 42., 43., 43., 23., 23., 31., 41., 51., 11.,
        52., 21.],
       [82., 55.,  0., 68., 46., 55., 23., 43., 41., 29., 79., 21., 64.,
        31., 51.],
       [46., 46., 68.,  0., 82., 15., 72., 31., 62., 42., 21., 51., 51.,
        43., 64.],
       [68., 42., 46., 82.,  0., 74., 23., 52., 21., 46., 82., 58., 46.,
        65., 23.],
       [52., 43., 55., 15., 74.,  0., 61., 23., 55., 31., 33., 37., 51.,
        29., 59.],
       [72., 43., 23., 72., 23., 61.,  0., 42., 23., 31., 77., 37., 51.,
        46., 33.],
       [42., 23., 43., 31., 52., 23., 42.,  0., 33., 15., 37., 33., 33.,
        31., 37.],
       [51., 23., 41., 62., 21., 55., 23., 33.,  0., 29., 62., 46., 29.,
        51., 11.],
       [55., 31., 29., 42., 46., 31., 31., 15., 29.,  0., 51., 21., 41.,
        23., 37.],
       [29., 41., 79., 21., 82., 33., 77., 37., 62., 51.,  0., 65., 42.,
       